In [4]:
!pip install requests


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
API_KEY = "a16322fa-a63d-49e5-8eed-aa2d6ee743ac"

url= "https://api.harvardartmuseums.org/classification"

import requests

params = {
    "apikey":API_KEY,
    "size":100
}

response =requests.get(url,params)
response

<Response [200]>

In [6]:
data = response.json()

In [7]:
data

{'info': {'totalrecordsperquery': 100,
  'totalrecords': 64,
  'pages': 1,
  'page': 1,
  'responsetime': '9 ms'},
 'records': [{'objectcount': 1,
   'name': 'Natural History Specimens',
   'id': 1133,
   'lastupdate': '2025-12-23T04:03:36-05:00',
   'classificationid': 1133},
  {'objectcount': 1,
   'name': 'Paintings with Text',
   'id': 197,
   'lastupdate': '2025-12-23T04:03:36-05:00',
   'classificationid': 197},
  {'objectcount': 66,
   'name': 'Performance Artifacts',
   'id': 224,
   'lastupdate': '2025-12-23T04:03:36-05:00',
   'classificationid': 224},
  {'objectcount': 1,
   'name': 'Text',
   'id': 204,
   'lastupdate': '2025-12-23T04:03:36-05:00',
   'classificationid': 204},
  {'objectcount': 80,
   'name': 'Tokens',
   'id': 492,
   'lastupdate': '2025-12-23T04:03:36-05:00',
   'classificationid': 492},
  {'objectcount': 349,
   'name': '(not assigned)',
   'id': 0,
   'lastupdate': '2025-12-23T04:03:36-05:00',
   'classificationid': 0},
  {'objectcount': 62,
   'name': 

In [8]:
obj= data["records"]

In [9]:
for i in obj:
    if i["objectcount"]>=2500:
        print(i["name"],":",i["objectcount"],":",i["classificationid"])

Accessories (non-art) : 2862 : 2
Photographs : 85836 : 17
Drawings : 33961 : 21
Prints : 72253 : 23
Paintings : 6863 : 26
Sculpture : 6643 : 30
Coins : 20270 : 50
Vessels : 6223 : 57
Textile Arts : 3446 : 62
Archival Material : 15737 : 64
Fragments : 5218 : 94
Manuscripts : 5001 : 185
Seals : 5899 : 189
Straus Materials : 5128 : 959


In [11]:
import requests
import time
url = "https://api.harvardartmuseums.org/object"

fragment_data = []

for page in range(1, 40):
    params = {
        "apikey": API_KEY,
        "size": 100,
        "page": page,
        "classification": 94   
    }

    response = requests.get(url, params=params)
    data = response.json()

    records = data.get("records", [])
    fragment_data.extend(records)

    print(f"Page {page} → Total records collected: {len(fragment_data)}")

    if len(fragment_data) >= 2500:
        break

    time.sleep(0.2)  

fragment_data = fragment_data[:2500]

print("Final count:", len(fragment_data))
    

Page 1 → Total records collected: 100
Page 2 → Total records collected: 200
Page 3 → Total records collected: 300
Page 4 → Total records collected: 400
Page 5 → Total records collected: 500
Page 6 → Total records collected: 600
Page 7 → Total records collected: 700
Page 8 → Total records collected: 800
Page 9 → Total records collected: 900
Page 10 → Total records collected: 1000
Page 11 → Total records collected: 1100
Page 12 → Total records collected: 1200
Page 13 → Total records collected: 1300
Page 14 → Total records collected: 1400
Page 15 → Total records collected: 1500
Page 16 → Total records collected: 1600
Page 17 → Total records collected: 1700
Page 18 → Total records collected: 1800
Page 19 → Total records collected: 1900
Page 20 → Total records collected: 2000
Page 21 → Total records collected: 2100
Page 22 → Total records collected: 2200
Page 23 → Total records collected: 2300
Page 24 → Total records collected: 2400
Page 25 → Total records collected: 2500
Final count: 2500


In [12]:
classifications = {
    "Fragments": 94,
    "Paintings": 26,
    "Coins": 50,
    "Drawings": 21,
    "Sculpture": 30
}

In [13]:
def fetch_objects(classification_id, limit=2500):
    all_data = []
    page = 1

    while len(all_data) < limit:
        params = {
            "apikey": API_KEY,
            "classification": classification_id,
            "size": 100,
            "page": page
        }

        response = requests.get(url, params=params)
        data = response.json()

        records = data.get("records", [])
        if not records:
            break

        all_data.extend(records)
        page += 1
        time.sleep(0.2)

    return all_data[:limit]

In [14]:
all_metadata = []
all_media = []
all_colors = []

for name, cid in classifications.items():
    print(f"Collecting {name}...")

    data = fetch_objects(cid)

    for obj in data:
        all_metadata.append({
            "id": obj.get("id"),
            "title": obj.get("title"),
            "culture": obj.get("culture"),
            "period": obj.get("period"),
            "century": obj.get("century"),
            "medium": obj.get("medium"),
            "dimensions": obj.get("dimensions"),
            "description": obj.get("description"),
            "department": obj.get("department"),
            "classification": obj.get("classification"),
            "accessionyear": obj.get("accessionyear"),
            "accessionmethod": obj.get("accessionmethod")
        })

        all_media.append({
            "objectid": obj.get("id"),
            "imagecount": obj.get("imagecount"),
            "mediacount": obj.get("mediacount"),
            "colorcount": obj.get("colorcount"),
            "rank": obj.get("rank"),
            "datebegin": obj.get("datebegin"),
            "dateend": obj.get("dateend")
        })

        for c in obj.get("colors", []):
            all_colors.append({
                "objectid": obj.get("id"),
                "color": c.get("color"),
                "spectrum": c.get("spectrum"),
                "hue": c.get("hue"),
                "percent": c.get("percent"),
                "css3": c.get("css3")
            })

In [15]:
import pymysql

conn = pymysql.connect(
    host="localhost",
    user="root",
    password="NewPassword123",
    
)

cursor = conn.cursor()
print("Connected successfully!")


Connected successfully!


In [16]:
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="NewPassword123",
    database="harvard_artifacts"
)

print("Connected successfully!")

Connected successfully!


In [17]:
cursor = conn.cursor()

cursor.execute("USE harvard_artifacts;")

0

In [18]:
metadata_sql = """
INSERT IGNORE INTO artifact_metadata (
    id, title, culture, period, century, medium, dimensions,
    description, department, classification, accessionyear, accessionmethod
) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

metadata_values = [
    (
        m["id"], m["title"], m["culture"], m["period"], m["century"],
        m["medium"], m["dimensions"], m["description"],
        m["department"], m["classification"],
        m["accessionyear"], m["accessionmethod"]
    )
    for m in all_metadata
]

cursor.executemany(metadata_sql, metadata_values)
conn.commit()

print("artifact_metadata inserted ✅")

artifact_metadata inserted ✅


In [19]:
media_sql = """
INSERT INTO artifact_media (
    objectid, imagecount, mediacount, colorcount, rank_value, datebegin, dateend
) VALUES (%s,%s,%s,%s,%s,%s,%s)
"""

media_values = [
    (
        m["objectid"], m["imagecount"], m["mediacount"],
        m["colorcount"], m["rank"],
        m["datebegin"], m["dateend"]
    )
    for m in all_media
]

cursor.executemany(media_sql, media_values)
conn.commit()

print("artifact_media inserted ✅")

artifact_media inserted ✅


In [20]:
color_sql = """
INSERT INTO artifact_colors (
    objectid, color, spectrum, hue, percent, css3
) VALUES (%s,%s,%s,%s,%s,%s)
"""

color_values = [
    (
        c["objectid"], c["color"], c["spectrum"],
        c["hue"], c["percent"], c["css3"]
    )
    for c in all_colors
]

cursor.executemany(color_sql, color_values)
conn.commit()

print("artifact_colors inserted ✅")

artifact_colors inserted ✅
